In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import glob,os
from sklearn.model_selection import train_test_split
import hashlib


MessageError: Error: credential propagation was unsuccessful

In [ ]:
#import spacy.cli
#spacy.cli.download("en_core_web_lg")

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/QE/Study2/TruthSeeker2023/Truth_Seeker_Model_Dataset_With_TimeStamps 1.xlsx').dropna()

# Creating unique identifiers for author-statement pairs
author_statement = df['author'] + "|" + df['statement']
df['root_node'] = pd.factorize(author_statement)[0]
df['root_node'] = df['root_node'].apply(lambda x:x+1)

# Calculate starting point for Tweet_IDs to avoid overlap
tweet_id_start = df['root_node'].max() + 1

# Assign unique identifiers for tweets, handling NaN values and ensuring no overlap
df['TWEET_ID'] = [i + tweet_id_start for i in range(len(df))]

df = df[['author', 'statement', 'BinaryNumTarget', 'tweet', 'timestamp', 'PARENT_ID', 'TWEET_ID']]

#given that it is a star-like graph without retweets,
origin = df[['author', 'statement',  'BinaryNumTarget', 'PARENT_ID']].drop_duplicates(['author','statement'])
origin['TWEET_ID']= origin['PARENT_ID']
origin['tweet']= origin['statement']

#here I will assume that the earliest timestamp for each PARENT_ID in the main DataFrame is the parentID time stamp
earliest_timestamps = df.groupby('PARENT_ID')['timestamp'].min().reset_index()

# Merge the earliest timestamp with the origin DataFrame on PARENT_ID
origin = origin.merge(earliest_timestamps, on='PARENT_ID', how='left')


df = pd.concat([df, origin], ignore_index=True)
df = df.sort_values(by='timestamp')

df

In [ ]:
df.columns

Index(['Unnamed: 0', 'author', 'statement', 'target', 'BinaryNumTarget',
       'manual_keywords', 'tweet', '5_label_majority_answer',
       '3_label_majority_answer', 'timestamp', 'ASP_ID', 'Tweet_ID'],
      dtype='object')

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix


from collections import defaultdict

from tqdm import tqdm
tqdm.pandas()

import numpy as np


In [ ]:
import spacy
from spacy import displacy

NER = spacy.load("en_core_web_lg")

In [ ]:
raw_text="The Indian Space Research Organisation or is the national space agency of India, headquartered in Bengaluru. It operates under Department of Space which is directly overseen by the Prime Minister of India while Chairman of ISRO acts as executive of DOS as well."
text1= NER(raw_text)
for word in text1.ents:
    print(word.text,word.label_)

The Indian Space Research Organisation ORG
India GPE
Bengaluru GPE
Department of Space ORG
India GPE
ISRO ORG


In [ ]:
def ner_extract(raw_text):
  try:
    text1= NER(raw_text)
    lst = [(word.text, word.label_) for word in text1.ents]
    return lst
  except:
    return []

ner_extract(raw_text)

[('The Indian Space Research Organisation', 'ORG'),
 ('India', 'GPE'),
 ('Bengaluru', 'GPE'),
 ('Department of Space', 'ORG'),
 ('India', 'GPE'),
 ('ISRO', 'ORG')]

In [ ]:
df['NER'] = df.tweet.progress_apply(lambda x: ner_extract(x))
df

100%|██████████| 135251/135251 [35:03<00:00, 64.31it/s]


,author,statement,BinaryNumTarget,tweet,timestamp,PARENT_ID,TWEET_ID,NER
134551,Jon Greenberg,"Says Donald Trump has ""changed his mind"" on ab...",1.0,"Says Donald Trump has ""changed his mind"" on ab...",Fri Apr 01 00:28:51 +0000 2016,358,358,"[(Donald Trump, PERSON)]"
43972,Jon Greenberg,"Says Donald Trump has ""changed his mind"" on ab...",1.0,@realDonaldTrump donald trump I so wanted to ...,Fri Apr 01 00:28:51 +0000 2016,358,45030,"[(@realDonaldTrump donald trump , PERSON)]"
41010,Lauren Carroll,"Says Donald Trump ""supports eminent domain"" an...",1.0,"@hilljobee Ah, so eminent domain that Trump su...",Fri Apr 01 00:52:15 +0000 2016,341,42068,"[(Trump, PERSON), (Heller, PERSON), (Cruz, PER..."
134534,Lauren Carroll,"Says Donald Trump ""supports eminent domain"" an...",1.0,"Says Donald Trump ""supports eminent domain"" an...",Fri Apr 01 00:52:15 +0000 2016,341,341,"[(Donald Trump, PERSON), (the Supreme Courts K..."
14968,Jill Terreri,"New York has passed ""the best paid family leav...",1.0,Paid family leave for New York kicks in in 2018.,Fri Apr 01 01:36:18 +0000 2016,122,16026,"[(New York, GPE), (2018, DATE)]"
...,...,...,...,...,...,...,...,...
5584,Louis Jacobson,In one of the states that determined the outco...,1.0,@Kokomothegreat TWO VOTES PER PRECINCT....I HA...,Wed Sep 30 23:16:07 +0000 2020,26,6642,"[(TWO, CARDINAL), (TWO, CARDINAL), (TWEETS, OR..."
38718,Sean Gorman,"Says President Barack Obama ""will not utter th...",1.0,Remember when Republicans hammered Obama becau...,Wed Sep 30 23:16:49 +0000 2020,330,39776,"[(Republicans, NORP), (Obama, PERSON), (Radica..."
69577,Sue Owen,"""Texas has the highest rate of uninsured in th...",1.0,@JohnCornyn Texas has the highest rate of unin...,Wed Sep 30 23:44:29 +0000 2020,539,70635,"[(@JohnCornyn, NORP), (Texas, GPE)]"
124792,Jon Greenberg,Says President Barack Obama spied on my campai...,0.0,"Corrupt Joe Biden and Obama, who spied on my c...",Wed Sep 30 23:55:01 +0000 2020,988,125850,"[(Joe Biden, PERSON), (Obama, PERSON)]"


In [ ]:
df.to_pickle('/content/drive/MyDrive/QE/Study2/TruthSeeker2023/Truth_Seeker_Model_Dataset_With_TimeStamps_NER.h5', compression='xz')

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/QE/Study2/TruthSeeker2023/Truth_Seeker_Model_Dataset_With_TimeStamps_NER.h5',compression='xz')

df = df.explode('NER')
df['target_type'] = df['NER'].apply(lambda x: x[1] if isinstance(x, tuple) else '')
df['target'] = df['NER'].apply(lambda x: x[0] if isinstance(x, tuple) else '')

target_types_to_retain = ['PERSON', 'ORG', 'GPE','']

df['target_filter'] = df.target_type.isin(target_types_to_retain)
df

,author,statement,BinaryNumTarget,tweet,timestamp,PARENT_ID,TWEET_ID,NER,target_type,target,target_filter
134551,Jon Greenberg,"Says Donald Trump has ""changed his mind"" on ab...",1.0,"Says Donald Trump has ""changed his mind"" on ab...",Fri Apr 01 00:28:51 +0000 2016,358,358,"(Donald Trump, PERSON)",PERSON,Donald Trump,True
43972,Jon Greenberg,"Says Donald Trump has ""changed his mind"" on ab...",1.0,@realDonaldTrump donald trump I so wanted to ...,Fri Apr 01 00:28:51 +0000 2016,358,45030,"(@realDonaldTrump donald trump , PERSON)",PERSON,@realDonaldTrump donald trump,True
41010,Lauren Carroll,"Says Donald Trump ""supports eminent domain"" an...",1.0,"@hilljobee Ah, so eminent domain that Trump su...",Fri Apr 01 00:52:15 +0000 2016,341,42068,"(Trump, PERSON)",PERSON,Trump,True
41010,Lauren Carroll,"Says Donald Trump ""supports eminent domain"" an...",1.0,"@hilljobee Ah, so eminent domain that Trump su...",Fri Apr 01 00:52:15 +0000 2016,341,42068,"(Heller, PERSON)",PERSON,Heller,True
41010,Lauren Carroll,"Says Donald Trump ""supports eminent domain"" an...",1.0,"@hilljobee Ah, so eminent domain that Trump su...",Fri Apr 01 00:52:15 +0000 2016,341,42068,"(Cruz, PERSON)",PERSON,Cruz,True
...,...,...,...,...,...,...,...,...,...,...,...
69577,Sue Owen,"""Texas has the highest rate of uninsured in th...",1.0,@JohnCornyn Texas has the highest rate of unin...,Wed Sep 30 23:44:29 +0000 2020,539,70635,"(@JohnCornyn, NORP)",NORP,@JohnCornyn,False
69577,Sue Owen,"""Texas has the highest rate of uninsured in th...",1.0,@JohnCornyn Texas has the highest rate of unin...,Wed Sep 30 23:44:29 +0000 2020,539,70635,"(Texas, GPE)",GPE,Texas,True
124792,Jon Greenberg,Says President Barack Obama spied on my campai...,0.0,"Corrupt Joe Biden and Obama, who spied on my c...",Wed Sep 30 23:55:01 +0000 2020,988,125850,"(Joe Biden, PERSON)",PERSON,Joe Biden,True
124792,Jon Greenberg,Says President Barack Obama spied on my campai...,0.0,"Corrupt Joe Biden and Obama, who spied on my c...",Wed Sep 30 23:55:01 +0000 2020,988,125850,"(Obama, PERSON)",PERSON,Obama,True


In [ ]:
df[df['target_filter']==True].to_pickle('/content/drive/MyDrive/QE/Study2/TruthSeeker2023/Truth_Seeker_Model_Dataset_With_TimeStamps__explode_NER.h5',compression='xz')


In [ ]:
df